In [1]:
#We need these in this file:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import numpy as np # linear algebra
import get_lodging_scores
import SVM_classifier_general as svm_general
import pickle
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
import glob

import sys
sys.path.append('cropping')
import load_read_name_extractor as lrne


In [2]:
data_paths_auto = glob.glob("Features/*npy")
size = int(len(data_paths_auto)/2)
features_paths = data_paths_auto[0:size]
names_paths = data_paths_auto[size:len(data_paths_auto)]

In [3]:
features_paths


['Features\\hog_features_(64, 128, 3)_cells_(10, 10)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(12, 12)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(14, 14)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(16, 16)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block

In [4]:
names_paths

['Features\\img_names_(64, 128, 3)_cells_(10, 10)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(12, 12)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(14, 14)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(16, 16)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\img_n

In [5]:
features_paths = [
'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(4, 4)_norm_L2_mir_True.npy']

In [6]:
names_paths = [
  'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(4, 4)_norm_L2_mir_True.npy']

In [7]:
#features_load = np.load(features_paths[0], allow_pickle=True)
#print(features_load.shape)
#print(features_load[0])
#features_paths[0]

In [8]:
#np.load(names_paths[0], allow_pickle=True) == np.load(names_paths[1], allow_pickle=True)

In [9]:
#features_load[3000].shape

In [10]:
def run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf", class_weights = None):
    for i in range (len(features_paths)):
                #laod features and names
        features_load = np.load(features_paths[0], allow_pickle=True)
        names_load = np.load(names_paths[0], allow_pickle=True)
        
        features_load = features_load.tolist()

        print(1)
        #load labels
        bin_size = 10
        labels = svm_general.load_labels(bin_size)

        #convert names into [flight_folder, ROI]
        names = lrne.it_name_extract_labels_from_img_jpeg(names_load)
        names_load = None
        print(2)
        #match labels with feature names
        features_load = svm_general.match_pic_label_to_names_new(features_load, labels, names)
        data_frame = features_load
        features_load = None
        labels = None
        names = None
        print(3)
        #shuffle data and convert to array
        np.random.shuffle(data_frame)

        #partition data
        percentage = 80
        #data_frame = np.array(data_frame)
        x_train, x_test, y_train, y_test = svm_general.partition_data_list(data_frame, percentage)
        data_frame = None
        features_aar = None

        #create svm
        clf = svm.SVC(kernel=svm_kernel, class_weight=class_weights, verbose=True)
        #fit model with training data
        clf.fit(x_train,y_train)
        #predict data
        y_pred = clf.predict(x_test)

        #Save the found model
        n_bins = int(np.floor(100/bin_size))
        path = "SVMC_" + "Kernel_" + svm_kernel + "_Nbins_" + str(n_bins) + "_ClassWeights_" + str(class_weights) + "_" + features_paths[i]
        svm_general.save_model(clf, path)

        #Save the results
        results = "Accuracy: "+str(accuracy_score(y_test, y_pred)) + "\n" + classification_report(y_test, y_pred)
        svm_general.save_results(results, path)

In [11]:
run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf")

1
2
3
Data Partitioned Successfully
Model Saved Succesfull
Results Saved Succesfull


c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(features_paths[0])
print(len(features_paths))